In [9]:
import numpy as np
import matplotlib.pyplot as plt

# ---------- Property correlations (from figs C8, C9) ----------

def k_nak(T):
    """
    Thermal conductivity of NaK, k_NaK(T)
    T in K
    Returns k in W/m-K

    Fig. C8: k = A + B*T, with
        A ≈ 0.218  (W/cm-K at T = 0)
        B ≈ 7.33e-5 (W/cm-K per K)
    Convert W/cm-K -> W/m-K by ×100.
    """
    A_k = 0.218
    B_k = 7.33e-5
    return (A_k + B_k * T) * 100.0  # W/m-K


def cp_nak(T):
    """
    Specific heat of NaK, cp(T)
    T in K
    Returns cp in J/kg-K

    Fig. C9: cp = A + B*T + C*T^2 + D*T^3  (J/g-K)
        A = 1.20
        B = -9.96e-4
        C = 9.66e-7
        D = -2.78e-10
    Convert J/g-K -> J/kg-K by ×1000.
    """
    A_cp = 1.20
    B_cp = -9.96e-4
    C_cp = 9.66e-7
    D_cp = -2.78e-10
    cp_g = A_cp + B_cp*T + C_cp*T**2 + D_cp*T**3   # J/g-K
    return cp_g * 1000.0                            # J/kg-K


# ---------- Geometry & flow (edit these for your case) ----------

# Inner and outer radii of the NaK annulus (m)
r_t = 0.001225   # tube (inner) radius
r_s = 0.001295   # shell (outer) radius

# Mass flow rate of NaK (kg/s)
m_dot = 0.0351

gamma = r_s / r_t

# Characteristic diameter 
D = 2.0 * (r_s - r_t)                           # m
A_NaK = np.pi * (r_s**2 - r_t**2)              # m^2

# Coefficients A, B, C for the h-correlation 
A_h = 4.63 + 0.686 * gamma
B_h = 0.215 + 4.3e-5 * gamma
C_h = 0.752 + 0.01657 * gamma - 8.83e-4 * gamma**2

# ---------- Temperature range and h(T) calculation ----------

T = (773+873)/2  # K

k = k_nak(T)          # W/m-K
cp = cp_nak(T)        # J/kg-K

# Peclet number (A4): Pe = m_dot * D * cp / (k * A_NaK)
Pe = m_dot * D * cp / (k * A_NaK)

# Convective heat-transfer coefficient (A1):
# h = k_NaK / D * [A + B * Pe^C]
h = k / D * (A_h + B_h * Pe**C_h)  # W/m^2-K

Nu = (A_h + B_h * Pe**C_h)
print(f"T = {T}")
print(f"Nu = {Nu}")
print(f"h = {h}")
print(f"Pe = {Pe}")
print(f"D_h = {D}")
print(f"A_f = {A_NaK}")

# ---------- Plot ----------
"""
plt.figure()
plt.plot(T, h)
plt.xlabel("NaK temperature, T (K)")
plt.ylabel("Convective heat-transfer coefficient, h (W/m$^2$·K)")
plt.title("NaK convective heat-transfer coefficient vs. temperature")
plt.grid(True)
plt.show()
"""


T = 823.0
Nu = 21.705675368234264
h = 4315179.737122597
Pe = 280.23969685506415
D_h = 0.00013999999999999993
A_f = 5.541769440932392e-07


'\nplt.figure()\nplt.plot(T, h)\nplt.xlabel("NaK temperature, T (K)")\nplt.ylabel("Convective heat-transfer coefficient, h (W/m$^2$·K)")\nplt.title("NaK convective heat-transfer coefficient vs. temperature")\nplt.grid(True)\nplt.show()\n'

In [8]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# NaK-77 (≈77.8 wt% K) thermophysical property correlations
# Based on regression to handbook-derived tabulated data
#   Source of tabulated points: "Sodium-NaK Engineering Handbook, Vol. 1"
#   as quoted in: https://www.creativeengineers.com/physical-properties-of-nak
#
# Validity (approximate):
#   Temperature: from melting point (~ -13 °C) up to ~400 °C
#   Composition: eutectic NaK-77/78
# Units:
#   T_C: degrees Celsius
#   rho: kg/m^3
#   mu: Pa·s
# -----------------------------------------------------------------------------

def nak77_density(T_C: np.ndarray) -> np.ndarray:
    """
    Density of liquid NaK-77 as a function of temperature (°C).

    ρ(T_C) = 945.3 - 0.22473 * T_C   [kg/m^3]
    (from ρ[g/cm^3] = 0.9453 - 2.2473e-4 * T_C, multiplied by 1000)
    """
    return 945.3 - 0.22473 * T_C


def nak77_viscosity(T_C: np.ndarray) -> np.ndarray:
    """
    Dynamic viscosity of liquid NaK-77 as a function of temperature (°C).

    Handbook eq. (1.19), valid for T > 400 °C:

        η = 0.082 * ρ^(1/3) * exp(979 * ρ / T)

    where:
        η  in cP
        ρ  in g/cm^3
        T  in K

    This function returns μ in Pa·s.
    """
    T_C = np.asarray(T_C, dtype=float)
    T_K = T_C + 273.15

    # density in g/cm^3
    rho_g = 0.9453 - 2.2473e-4 * T_C

    # viscosity in cP
    mu_cP = 0.082 * rho_g**(1.0 / 3.0) * np.exp(979.0 * rho_g / T_K)

    # convert cP -> Pa·s
    mu_Pa_s = mu_cP * 1.0e-3

    return mu_Pa_s


if __name__ == "__main__":
    # Temperature range of interest: below 400 °C
    T_C = 550

    rho = nak77_density(T_C)      # kg/m^3
    mu = nak77_viscosity(T_C)     # Pa·s

"""
    # -------------------------------------------------------------------------
    # Plot viscosity and density vs temperature
    # -------------------------------------------------------------------------
    fig, ax1 = plt.subplots(figsize=(6, 4))

    ax1.set_xlabel("Temperature $T$ [°C]")
    ax1.set_ylabel(r"Viscosity $\mu$ [Pa·s]")
    ax1.semilogy(T_C, mu, label="NaK-77 viscosity")
    ax1.grid(True, which="both", linestyle="--", linewidth=0.5)

    ax2 = ax1.twinx()
    ax2.set_ylabel(r"Density $\rho$ [kg/m$^3$]")
    ax2.plot(T_C, rho, linestyle="-", label="NaK-77 density")

    plt.title("NaK-77 properties vs temperature (T < 400 °C)")
    fig.tight_layout()
    plt.show()
"""

print(f"mu = {mu}")

mu = 0.00020408413792388032


In [11]:
def reynolds_number(mdot, D, A, mu):
    """
    Compute Reynolds number for internal flow using:
        Re = (m_dot * D) / (A * mu)

    Parameters
    ----------
    mdot : float
        Mass flow rate [kg/s]
    D : float
        Hydraulic diameter or characteristic length [m]
    A : float
        Flow cross-sectional area [m^2]
    mu : float
        Dynamic viscosity [Pa·s]

    Returns
    -------
    Re : float
        Reynolds number [-]
    """
    return mdot * D / (A * mu)


# example
if __name__ == "__main__":
    m_dot = 0.0351      # kg/s
    D_h = 0.00013999       # m
    A_f = 5.54176e-7        # m^2
    mu = 0.00020408       # Pa·s

    Re = reynolds_number(m_dot, D_h, A_f, mu)
    print("Re =", Re)


Re = 43446.61753049885
